In [1]:
from IPython import get_ipython
from IPython.core.display import display
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
get_ipython().run_line_magic('run', 'setup')

In [2]:
#leeftijdsgroepen = SimpleNamespace()
@run
def cell():
  global rivm
  rivm = RIVM.csv('COVID-19_casus_landelijk')
  display(rivm.head())

downloading rivm/COVID-19_casus_landelijk-2022-01-11@14-15.csv


rivm COVID-19_casus_landelijk zipping rivm/COVID-19_casus_landelijk-2022-01-11@14-15.csv


loading rivm/COVID-19_casus_landelijk-2022-01-11@14-15.csv.gz


,Date_file,Date_statistics,Date_statistics_type,Agegroup,Sex,Province,Hospital_admission,Deceased,Week_of_death,Municipal_health_service
0,2022-01-11 10:00:00,2020-01-01,DOO,40-49,Female,Noord-Holland,No,No,NaN,GGD Amsterdam
1,2022-01-11 10:00:00,2020-01-01,DOO,50-59,Male,Gelderland,No,No,NaN,Veiligheids- en Gezondheidsregio Gelderland-Mi...
2,2022-01-11 10:00:00,2020-01-01,DOO,20-29,Female,Zuid-Holland,No,No,NaN,GGD Hollands-Midden
3,2022-01-11 10:00:00,2020-01-01,DOO,60-69,Female,Noord-Holland,No,No,NaN,GGD Hollands-Noorden
4,2022-01-11 10:00:00,2020-01-04,DOO,10-19,Female,Gelderland,Unknown,No,NaN,GGD Gelderland-Zuid


In [3]:
@run
def cell():
  global bevolking

  # December 2021 als backup:
  aantallenpercohort = {'0-9': 1757736,
    '10-19': 1981885,
    '20-29': 2263186,
    '30-39': 2213705,
    '40-49': 2137833,
    '50-59': 2550358,
    '60-69': 2171183,
    '70-79': 1649291,
    '80-89': 722027,
    '90+': 135289}

  global bevolking

  bevolking = None
  try:
    bevolking = CBS.bevolking(leeftijdsgroepen=True)
    bevolking.to_csv("cbs/leeftijdsgroepen_cbs.csv")
  except Exception as e:
    print(e)
    # gebruik laatste csv
    try:
      bevolking = pd.read_csv("cbs/leeftijdsgroepen_cbs.csv")
      bevolking.set_index('Range', inplace=True)
    except Exception as ee:
      print(ee)
      bevolking = None

  if bevolking is None:
    bevolking = aantallenpercohort

In [4]:
# Bereken de stand van zaken van besmettingen / hospitalisaties / overlijden, per cohort in absolute aantallen en aantallen per 100k, met een kleur indicator voor de aantallen.
# vervang <50 en Unknown door Onbekend
@run
def cell():
  rivm['Cohort'] = rivm['Agegroup'].replace({'<50': 'Onbekend', 'Unknown': 'Onbekend'})
  # aangenomen 'gemiddelde' leeftijd van een cohort: minimum waarde + 5
  assumed_cohort_age = [(cohort, [int(n) for n in cohort.replace('+', '').split('-')]) for cohort in rivm['Cohort'].unique() if cohort[0].isdigit()]
  assumed_cohort_age = { cohort: min(rng) + 5 for cohort, rng in assumed_cohort_age }
  rivm['Gemiddelde leeftijd'] = rivm['Cohort'].apply(lambda x: assumed_cohort_age.get(x, np.nan))

  # verwijder tijd
  rivm['Date_file_date'] = pd.to_datetime(rivm['Date_file'].replace(r' .*', '', regex=True))

  rivm['Date_statistics_date'] = pd.to_datetime(rivm['Date_statistics'])

  # weken terug = verschil tussen Date_file en Date_statistcs, gedeeld door 7 dagen
  rivm['Weken terug'] = np.floor((rivm['Date_file_date'] - rivm['Date_statistics_date'])/np.timedelta64(7, 'D')).astype(int)

  # voeg key, gem leeftijd, kleurnummer en totaal toe
  Date_file = rivm['Date_file_date'].unique()[0].astype('M8[D]').astype('O')
  # check type bevolking
  if type(bevolking) == dict:
    cohorten = list(bevolking.keys())
  else:
    cohorten = list(bevolking.index)
  cohorten += ['Onbekend']

  def summarize(df, category, prefix):
    # aangezien we hier de dataframe in-place wijzigen (bijv door toevoegen kolommen)
    # en we het 'rivm' frame later nog clean nodig hebben
    df = df.copy(deep=True)

    df = (df
          .groupby(['Weken terug', 'Cohort'])['count']
          .sum()
          .unstack(fill_value=np.nan)
          .reset_index()
          .rename_axis(None, axis=1)
        ).merge(df
          # we voegen hier gemiddelde leeftijd toe, want die willen we op een ander
          # niveau aggregeren voor 'df' overschreven word
          .groupby(['Weken terug'])['Gemiddelde leeftijd']
          .mean()
          .to_frame(), on='Weken terug'
        )

    # altijd 52 rijen
    df = pd.Series(np.arange(52), name='Weken terug').to_frame().merge(df, how='left', on='Weken terug')

    # toevoegen missende cohorten
    for col in cohorten:
      if not col in df:
        df[col] = np.nan

    # sommeer per rij (axis=1) over de cohorten om een totaal te krijgen
    df['Totaal'] = df[cohorten].sum(axis=1)

    # voeg periode en datum toe
    # periode afgeleid van weken-terug (= de index voor deze dataframe)
    df['Datum'] = pd.to_datetime(Date_file)
    df['Periode'] = (df
      .index.to_series()
      .apply(
        lambda x: (
          (Date_file + datetime.timedelta(weeks=-(x+1), days=1)).strftime('%d/%m')
          + '-'
          + (Date_file + datetime.timedelta(weeks=-x)).strftime('%d/%m')
        )
      )
    )

    # voeg 'Key' en 'Type' kolom toe. Variabele 'type' kan niet, is een language primitive.
    df['Key'] = prefix + df.index.astype(str).str.rjust(3, fillchar='0')
    df['Type'] = category

    # voeg de kleur kolommen toe
    for col in cohorten:
      df['c' + col] = ((df[col] / df[[col for col in cohorten]].max(axis=1)) * 1000).fillna(0).astype(int)

    # herschikken van de kolommen
    colorder = ['Key', 'Weken terug', 'Datum', 'Periode', 'Gemiddelde leeftijd', 'Totaal', 'Type']
    return df[colorder + [col for col in df if col not in colorder]]

  if type(bevolking) == dict:
    factor = {}
    for c in bevolking: factor[c] = 100_000 / bevolking[c]
  else:
    factor = bevolking.to_dict()['per 100k']

  global tabel
  tabel = pd.concat(
    # flatten the result list zodat pd.concat ze onder elkaar kan plakken
    functools.reduce(lambda a, b: a + b, [
      [summarize(df.assign(count=1), label, prefix), summarize(df.assign(count=df['Cohort'].apply(lambda x: factor.get(x, np.nan))), label + ' per 100.000', prefix + '100k')]
      for df, label, prefix in [
        (rivm, 'Positief getest', 'p'), # volledige count per cohort
        (rivm[rivm.Hospital_admission == 'Yes'], 'Ziekenhuisopname', 'h'), # count van cohort voor Hospital_admission == 'Yes'
        (rivm[rivm.Deceased == 'Yes'], 'Overleden', 'd'), # count van cohort voor Deceased == 'Yes'
      ]
    ])
  )

  if type(bevolking) == dict:
    totale_bevolking = sum(bevolking.values())
    print(totale_bevolking)
  else:
    totale_bevolking_per_cohort = bevolking.to_dict()['BevolkingOpDeEersteVanDeMaand']
    totale_bevolking = sum( totale_bevolking_per_cohort.values())

  keys = [ key for key in tabel["Key"]]
  totals = [ x for x in tabel["Totaal"]]
  for k in range(0, len(keys)):
    key = keys[k]
    if '100k' in key:
      abskey = key[0] + key[5:]
      kk = keys.index(abskey)
      if isinstance(totals[k], float):
        correctedtotal = totals[kk] * (100_000 / totale_bevolking)
        # print([k, totals[k], totals[kk], correctedtotal])
        totals[k] = correctedtotal

  tabel["Totaal"] = totals


  # rood -> groen
  cdict = {
    'red':   ((0.0, 0.0, 0.0),   # no red at 0
              (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
              (1.0, 0.8, 0.8)),  # set to 0.8 so its not too bright at 1
    'green': ((0.0, 0.8, 0.8),   # set to 0.8 so its not too bright at 0
              (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
              (1.0, 0.0, 0.0)),  # no green at 1
    'blue':  ((0.0, 0.0, 0.0),   # no blue at 0
              (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
              (1.0, 0.0, 0.0))   # no blue at 1
  }
  cm = colors.LinearSegmentedColormap('GnRd', cdict)
  # geel -> paars
  cm = sns.color_palette('viridis_r', as_cmap=True)
  display(tabel
    .fillna(0)
    .head()
    .round(1)
    .reset_index(drop=True)
    .style.background_gradient(cmap=cm, axis=1, subset=cohorten)
  )

,Key,Weken terug,Datum,Periode,Gemiddelde leeftijd,Totaal,Type,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+,Onbekend,c0-9,c10-19,c20-29,c30-39,c40-49,c50-59,c60-69,c70-79,c80-89,c90+,cOnbekend
0,p000,0,2022-01-11 00:00:00,05/01-11/01,32.900000,132033.000000,Positief getest,5877.000000,23882.000000,40512.000000,20030.000000,17108.000000,16828.000000,5322.000000,1728.000000,576.000000,170.000000,0.000000,145,589,1000,494,422,415,131,42,14,4,0
1,p001,1,2022-01-11 00:00:00,29/12-04/01,34.700000,135109.000000,Positief getest,6099.000000,20630.000000,39560.000000,21815.000000,16870.000000,17744.000000,7842.000000,3343.000000,972.000000,233.000000,1.000000,154,521,1000,551,426,448,198,84,24,5,0
2,p002,2,2022-01-11 00:00:00,22/12-28/12,37.200000,89479.000000,Positief getest,7186.000000,10818.000000,17711.000000,15910.000000,13195.000000,12477.000000,7533.000000,3413.000000,993.000000,242.000000,1.000000,405,610,1000,898,745,704,425,192,56,13,0
3,p003,3,2022-01-11 00:00:00,15/12-21/12,35.800000,88104.000000,Positief getest,11338.000000,12321.000000,13629.000000,15015.000000,12791.000000,10224.000000,7131.000000,3773.000000,1450.000000,431.000000,1.000000,755,820,907,1000,851,680,474,251,96,28,0
4,p004,4,2022-01-11 00:00:00,08/12-14/12,36.400000,105296.000000,Positief getest,15049.000000,15606.000000,12871.000000,17012.000000,15713.000000,11572.000000,8930.000000,5275.000000,2513.000000,752.000000,3.000000,884,917,756,1000,923,680,524,310,147,44,0


In [5]:
if knack:
  await knack.publish(tabel.fillna(0).assign(Datum=tabel.Datum.dt.strftime('%Y-%m-%d')), 'Leeftijdsgroep', Cache)

infinities:


Empty DataFrame
Columns: []
Index: []


nan:


Empty DataFrame
Columns: []
Index: []


Key                     object
Weken terug              int64
Datum                   object
Periode                 object
Gemiddelde leeftijd    float64
Totaal                 float64
Type                    object
0-9                    float64
10-19                  float64
20-29                  float64
30-39                  float64
40-49                  float64
50-59                  float64
60-69                  float64
70-79                  float64
80-89                  float64
90+                    float64
Onbekend               float64
c0-9                     int64
c10-19                   int64
c20-29                   int64
c30-39                   int64
c40-49                   int64
c50-59                   int64
c60-69                   int64
c70-79                   int64
c80-89                   int64
c90+                     int64
cOnbekend                int64
dtype: object
updating knack


...

updating: [{'Key': 1, 'Updating Leeftijdsgroep': 'false'}]
Key                         int64
Updating Leeftijdsgroep    object
dtype: object


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.11s/it]

100%|██████████| 1/1 [00:01<00:00,  1.11s/it]


rate limit: 7 
LaatsteUpdate API calls: create: 0, read: 0, update: 1, delete: 0, backoff: 0
